# Instalaciones e importaciones previas

## Instalaciones

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [2]:
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.2MB/s eta 0:04:22tcmalloc: large alloc 1147494400 bytes == 0x55f0326e6000 @  0x7f89f5ec5615 0x55eff9f4a06c 0x55effa029eba 0x55eff9f4ce8d 0x55effa03e99d 0x55eff9fc0fe9 0x55eff9fbbb0e 0x55eff9f4e77a 0x55eff9fc0e50 0x55eff9fbbb0e 0x55eff9f4e77a 0x55eff9fbd86a 0x55effa03f7c6 0x55eff9fbcee2 0x55effa03f7c6 0x55eff9fbcee2 0x55effa03f7c6 0x55eff9fbcee2 0x55effa03f7c6 0x55effa0c1431 0x55effa022049 0x55eff9f8cc84 0x55eff9f4d8e9 0x55eff9fc1ade 0x55eff9f4e69a 0x55eff9fbca45 0x55eff9fbbe0d 0x55eff9f4e77a 0x55eff9fbca45 0x55eff9f4e69a 0x55eff9fbca45
     |█████████████████████████████▏  | 1055.7MB 1.3MB/s eta 0:01:19tcmalloc: large alloc 1434370048 bytes == 0x55f076d3c000 @  0x7f89f5ec5615 0x55eff9f4a06c 0x55effa029eba 0x55eff9f4ce8d 0x55effa03e99d 0x55eff9fc0fe9 0x55eff9fbbb0e 0x55eff9f4e77a 0x55eff9fc0e50 0x55eff9fbbb0e 0x55eff9f4e77a 0x55eff9fbd86a 0x55effa03f7c6 0x55eff9fbcee2 0

In [3]:
!pip install --upgrade gensim mlflow tqdm

     |████████████████████████████████| 24.2MB 136kB/s 
     |████████████████████████████████| 14.2MB 250kB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 163kB 59.8MB/s 
     |████████████████████████████████| 1.1MB 35.8MB/s 
     |████████████████████████████████| 153kB 56.6MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.1-cp37-none-any.whl size=17159 sha256=fa13f55f9842549c82163f6ef11d85362b02d76a89a27fd57c5e5c67bd291b6a
  Stored in directory: /root/.cache/pip/wheels/b4/1f/b8/66bd9bc3a9d6c6987ff6c4dfeb6f1fe97b5a0e5ed5849c0437
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=ee9

## Librerías

In [4]:
import numpy as np
import mlflow
import csv
import json
import gzip
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

## Carga del Dataset

In [5]:
%%bash

mkdir -p data
curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
curl -L http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2
tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
bunzip2 ./data/SBW-vectors-300-min5.txt.bz2

meli-challenge-2019/
meli-challenge-2019/spanish.test.jsonl.gz
meli-challenge-2019/portuguese.validation.jsonl.gz
meli-challenge-2019/portuguese.train.jsonl.gz
meli-challenge-2019/spanish.train.jsonl.gz
meli-challenge-2019/spanish_token_to_index.json.gz
meli-challenge-2019/portuguese_token_to_index.json.gz
meli-challenge-2019/spanish.validation.jsonl.gz
meli-challenge-2019/portuguese.test.jsonl.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  945M  100  945M    0     0  17.0M      0  0:00:55  0:00:55 --:--:-- 17.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   259  100   259    0     0    315      0 --:--:-- --:--:-- --:--:--   315
100  780M  100  780M    0     0  17.0M      0  0:00:45  0:00:45 --:--:-- 18.3M


In [6]:
# transforma archivos comprimidos json en .csv
def json_to_csv(file_path, file_name, new_file_name):
  d = {'title': [], 'category': []}
  with gzip.open(file_path+file_name, "rt") as corpus:
    for idx, line in enumerate(tqdm(corpus)):
        df = json.loads(line) 
        d['title'].append(df['title'])
        d['category'].append(df['category']) 
  df = pd.DataFrame(data=d)
  df.to_csv(file_path+new_file_name+'.csv', index=False)
  
# generamos un data set corto para hacer pruebas rápidas
  df[df.index < 10000].to_csv(file_path+new_file_name+'_short.csv', index=False) 

In [7]:
json_to_csv('./data/meli-challenge-2019/', 
            'spanish.train.jsonl.gz',
            'spanish_train')

json_to_csv('./data/meli-challenge-2019/', 
            'spanish.validation.jsonl.gz',
            'spanish_validation')

json_to_csv('./data/meli-challenge-2019/', 
            'spanish.test.jsonl.gz',
            'spanish_test')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
rm -r ./data/meli-challenge-2019/*.jsonl.gz

In [9]:
len_train = !cat ./data/meli-challenge-2019/spanish_train.csv | wc -l
len_train = int(len_train[0])
print('longitud dataset de entrenamiento: ', len_train)

len_train_short = !cat ./data/meli-challenge-2019/spanish_train_short.csv | wc -l
len_train_short = int(len_train_short[0])
print('longitud dataset corto de entrenamiento: ', len_train_short)

longitud dataset de entrenamiento:  4895281
longitud dataset corto de entrenamiento:  10001


In [10]:
len_validation = !cat ./data/meli-challenge-2019/spanish_validation.csv | wc -l
len_validation = int(len_validation[0])
print('longitud dataset de validación: ', len_validation)

len_validation_short = !cat ./data/meli-challenge-2019/spanish_validation_short.csv | wc -l
len_validation_short = int(len_validation_short[0])
print('longitud dataset corto de validación: ', len_validation_short)

longitud dataset de validación:  1223821
longitud dataset corto de validación:  10001


In [11]:
len_test = !cat ./data/meli-challenge-2019/spanish_test.csv | wc -l
len_test = int(len_test[0])
print('longitud dataset corto de test: ', len_test)

len_test_short = !cat ./data/meli-challenge-2019/spanish_test_short.csv | wc -l
len_test_short = int(len_test_short[0])
print('longitud dataset corto de test: ', len_test_short)

longitud dataset corto de test:  63681
longitud dataset corto de test:  10001


# Preprocesamiento

In [12]:
import random

class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 dataset_size,
                 random_buffer_size=2048,
                 transform=None):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.dataset_size = dataset_size
        self.random_buffer_size = random_buffer_size
        self.transform = transform
    
    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []
                csv_reader = csv.reader(dataset)
                next(csv_reader)

                for line in csv_reader:
                    title, category = line

                    if self.random_buffer_size == 1:
                        item = {
                            "title": f"{title}",
                            "category": category
                        }
                        if self.transform:
                            item = self.transform(item)
                        yield item
                    else:
                        shuffle_buffer.append({
                            "title": f"{title}",
                            "category": category
                        })

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                if self.transform:
                                    item = self.transform(item)
                                yield item
                            shuffle_buffer = []

            if len(shuffle_buffer) > 0:
                random.shuffle(shuffle_buffer)
                for item in shuffle_buffer:
                    if self.transform:
                        item = self.transform(item)
                    yield item
        except GeneratorExit:
            return

In [13]:
class RawDataProcessor:
    def __init__(self, 
                 dataset_path,
                 dataset_size,
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary()
        labels = set()

        with open(dataset_path, "rt") as dataset:
            csv_reader = csv.reader(dataset)
            if ignore_header:
              next(csv_reader)
            for line in tqdm(csv_reader, total=dataset_size):
                title, category = line
                self.dictionary.add_documents(
                    [self._preprocess_string(f"{title}")]
                )
                labels.add(category)
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        }),
        self.idx_to_target = sorted(labels)
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target, start=1)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx.get(target, 0)
    
    def __call__(self, item):
        if isinstance(item, list):
          data = [self.encode_data(i["title"]) for i in item]
          target = [self.encode_target(i["category"]) for i in item]
        else:
          data = self.encode_data(item["title"])
          target = self.encode_target(item["category"])
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos: dataset completo

Leemos el dataset completo de MeLiChallenge y lo dividimos en subconjuntos de entrenamiento y evaluación.

In [14]:
preprocess = RawDataProcessor(
    "./data/meli-challenge-2019/spanish_train.csv",
    len_train
)

  0%|          | 0/4895281 [00:00<?, ?it/s]

In [15]:
train_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_train.csv",
    len_train,
    random_buffer_size=2048,
    transform=preprocess
)

validation_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_validation.csv",
    len_validation,
    random_buffer_size=1,
    transform=preprocess
)

test_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_test.csv",
    len_test,
    random_buffer_size=1,
    transform=preprocess
)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 4895281 training elements and 1223821 validation elements
Sample train element:
{'data': [3341, 2339, 705, 1717], 'target': 295}


## Lectura de datos: dataset corto (lo utilizamos para hacer pruebas antes de la versión final)

Leemos el dataset corto de MeLiChallenge y lo dividimos en subconjuntos de entrenamiento y evaluación.
  

In [ ]:
# datasets cortos
preprocess = RawDataProcessor(
    "./data/meli-challenge-2019/spanish_train_short.csv",
    len_train_short
)

  0%|          | 0/10001 [00:00<?, ?it/s]

In [ ]:
# datasets cortos
train_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_train_short.csv",
    len_train_short,
    random_buffer_size=2048,
    transform=preprocess
)

validation_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_validation_short.csv",
    len_validation_short,
    random_buffer_size=1,
    transform=preprocess
)

test_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish_test_short.csv",
    len_test_short,
    random_buffer_size=1,
    transform=preprocess
)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 10001 training elements and 10001 validation elements
Sample train element:
{'data': [2184, 325, 2182, 2183], 'target': 548}


## Collation function: modelo de perceptrón multicapa

In [16]:
# Para MLP

class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

## DataLoaders: modelo de perceptrón multicapa (MLP)

In [ ]:
# Para MLP
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)

In [ ]:
next(iter(train_loader))

{'data': tensor([[2843,  695, 3745,  ...,    0,    0,    0],
         [3078, 3079, 3077,  ...,    0,    0,    0],
         [3204, 2198, 3206,  ...,  776,    0,    0],
         ...,
         [1247, 1246, 1248,  ...,    0,    0,    0],
         [ 184, 1751, 2126,  ...,    0,    0,    0],
         [ 772,  587, 1463,  ...,    0,    0,    0]]),
 'target': tensor([ 85., 324., 549.,  82., 520., 542., 432., 133., 364., 515., 221.,  68.,
         221., 518., 457., 280., 151.,  24., 195., 368., 171., 247., 223.,  64.,
         234., 483., 580., 208., 450., 221., 156., 127., 295.,  16., 300., 194.,
         236., 155., 382., 295., 394.,  21., 451., 624., 572., 468., 129., 596.,
         182., 423.,  43., 539.,  23., 581., 126., 423., 619.,  21., 194.,  14.,
         133., 611., 575., 615., 408., 612., 501., 151., 577.,  85., 557., 581.,
         571., 244., 475., 439., 400., 386., 330., 435., 501.,   4.,   3., 241.,
         386.,   8., 170., 316., 574., 428., 158., 260., 398.,  41., 216., 470.,


## Modelo de clasificación: perceptrón multicapa (MLP)

In [ ]:
# Para MLP

class MeliChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 n_labels,
                 vector_size,
                 freeze_embedings,
                 hidden1=128,
                 hidden2=128,
                 dropout=0.3,
                 seed=1
                 ):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, hidden1)
        self.hidden2 = nn.Linear(hidden1, hidden2)
        self.output = nn.Linear(hidden2, n_labels)
        self.vector_size = vector_size
        self.dropout = dropout
        self.seed = np.random.seed(seed)
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        if self.dropout:
          x = F.dropout(x, self.dropout)
        x = F.relu(self.hidden2(x))
        if self.dropout:
          x = F.dropout(x, self.dropout)
        x = self.output(x)
        return x

# Exploración de hiperparámetros: perceptrón multicapa (MLP)



In [ ]:
h1 = np.array([64, 128, 512, 2048])
h2 = np.array([64, 128, 512, 2048])
d = np.array([0.1, 0.3, 0.6])
l = np.array([1e-1, 1e-3, 1e-6, 1e-10])
w = np.array([1e-1, 1e-5, 1e-10])

for i in trange(5):
  hidden1 = int(np.random.choice(h1, 1)) 
  hidden2 = int(np.random.choice(h2, 1))
  dropout = float(np.random.choice(d, 1))
  lr = float(np.random.choice(l, 1))
  wd = float(np.random.choice(w, 1))

  mlflow.set_experiment("Random search_mlp"+str(i))
  with mlflow.start_run():
      mlflow.log_param("model_name", "mlp")
      mlflow.log_param("freeze_embedding", True)
      mlflow.log_params({
          "embedding_size": 300,
          "hidden1_size": (300, hidden1),
          "hidden2_size": (hidden1, hidden2),
          'dropout': dropout,
          'regularización': lr,
          'weight decay': wd
      })
      model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                      preprocess.dictionary, 
                                      len(preprocess.target_to_idx) + 1,
                                      300, 
                                      True)
      device = torch.device("cuda")
      model = model.to(device)
      loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd) 
      for epoch in trange(3):
          model.train()
          running_loss = []
          for idx, batch in enumerate(tqdm(train_loader)):
              optimizer.zero_grad()
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              loss_value = loss(output, target)
              loss_value.backward()
              optimizer.step()
              running_loss.append(loss_value.item())        
          mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
          
          model.eval()
          running_loss = []
          targets = []
          predictions = []
          for batch in tqdm(validation_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              running_loss.append(
                  loss(output, target).item()
              )
              targets.extend(batch["target"].numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
      
      with tempfile.TemporaryDirectory() as tmpdirname:
          targets = []
          predictions = []
          for batch in tqdm(test_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              targets.extend(target.cpu().detach().numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
              f"{tmpdirname}/predictions.csv.gz", index=False
          )
          mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)
          mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/5 [00:00<?, ?it/s]

INFO: 'Random search_mlp0' does not exist. Creating a new experiment


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

...

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

# Experimento con mlflow: perceptrón multicapa (MLP)

In [ ]:
# hiperparámetros seleccionados a partir del random search
for i in range(3):  
  hidden1 = 2048
  hidden2 = 2048
  dropout = 0.1
  lr = 1e-6
  wd = 1e-5
  s = i

  mlflow.set_experiment("Clasificación_con_mlp_rep"+str(i))
  with mlflow.start_run():
      mlflow.log_param("model_name", "mlp")
      mlflow.log_param("freeze_embedding", True)
      mlflow.log_params({
          "embedding_size": 300,
          "hidden1_size": (300, hidden1),
          "hidden2_size": (hidden1, hidden2),
          'dropout': dropout,
          'regularización': lr,
          'weight decay': wd
      })
      model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                      preprocess.dictionary, 
                                      len(preprocess.target_to_idx) + 1,
                                      300, 
                                      True,
                                      seed=s
                                      )
      device = torch.device("cuda")
      model = model.to(device)
      loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd) 
      for epoch in trange(20):
          model.train()
          running_loss = []
          for idx, batch in enumerate(tqdm(train_loader)):
              optimizer.zero_grad()
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              loss_value = loss(output, target)
              loss_value.backward()
              optimizer.step()
              running_loss.append(loss_value.item())        
          mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
          
          model.eval()
          running_loss = []
          targets = []
          predictions = []
          for batch in tqdm(validation_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              running_loss.append(
                  loss(output, target).item()
              )
              targets.extend(batch["target"].numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
      
      with tempfile.TemporaryDirectory() as tmpdirname:
          targets = []
          predictions = []
          for batch in tqdm(test_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              targets.extend(target.cpu().detach().numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
              f"{tmpdirname}/predictions.csv.gz", index=False
          )
          mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)
          mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

...

  0%|          | 0/498 [00:00<?, ?it/s]

INFO: 'Clasificación_con_mlp_rep1' does not exist. Creating a new experiment


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
"""
Dado que las métricas obtenidas con los valores de base fueron mejores
que las obtenidas con los mejores hiperparámetros obtenidos con la búsqueda aleatoria
decidimos volver a correr el experimento la misma cantidad de veces pero con los hiperparámetros 
de base para compararlos con los resultados anteriores
"""
for i in range(1):
  # hiperparámetros seleccionados a partir del random search
  hidden1 = 128
  hidden2 = 128
  dropout = 0.3
  lr = 1e-3
  wd = 1e-5
  s = i

  mlflow.set_experiment("Clasificación_hiperparámetros_base_mlp_rep"+str(i))
  with mlflow.start_run():
      mlflow.log_param("model_name", "mlp")
      mlflow.log_param("freeze_embedding", True)
      mlflow.log_params({
          "embedding_size": 50,
          "hidden1_size": (300, hidden1),
          "hidden2_size": (hidden1, hidden2),
          'dropout': dropout,
          'regularización': lr,
          'weight decay': wd
      })
      model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                      preprocess.dictionary, 
                                      len(preprocess.target_to_idx) + 1,
                                      300, 
                                      True,
                                      seed=s
                                      )
      device = torch.device("cuda")
      model = model.to(device)
      loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd) 
      for epoch in trange(10):
          model.train()
          running_loss = []
          for idx, batch in enumerate(tqdm(train_loader)):
              optimizer.zero_grad()
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              loss_value = loss(output, target)
              loss_value.backward()
              optimizer.step()
              running_loss.append(loss_value.item())        
          mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
          
          model.eval()
          running_loss = []
          targets = []
          predictions = []
          for batch in tqdm(validation_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              running_loss.append(
                  loss(output, target).item()
              )
              targets.extend(batch["target"].numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
      
      with tempfile.TemporaryDirectory() as tmpdirname:
          targets = []
          predictions = []
          for batch in tqdm(test_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              targets.extend(target.cpu().detach().numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
              f"{tmpdirname}/predictions.csv.gz", index=False
          )
          mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)
          mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

...

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

## Collation function: modelo de red convolucional (CNN)

In [17]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

## DataLoaders: modelo de red convolucional (CNN)

In [18]:
FILTERS_LENGTH =[2, 3, 4]
FILTERS_COUNT=100
pad_sequences = PadSequences(min_length=max(FILTERS_LENGTH))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)

## Modelo de clasificación: red convolucional (CNN)

In [20]:
class MeliChallengeClassifier_CNN(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 n_labels,
                 vector_size,
                 freeze_embedings,
                 FILTERS_COUNT= FILTERS_COUNT,
                 FILTERS_LENGTH= FILTERS_LENGTH,
                 dropout= 0.3,
                 seed= 1
                 ):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.convs = []
        for filter_lenght in FILTERS_LENGTH:
            self.convs.append(
                nn.Conv1d(vector_size, FILTERS_COUNT, filter_lenght)
            )
        self.convs = nn.ModuleList(self.convs)
        self.fc = nn.Linear(FILTERS_COUNT * len(FILTERS_LENGTH), 128)
        self.output = nn.Linear(128, n_labels)
        self.vector_size = vector_size
        self.dropout = dropout
        self.seed = np.random.seed(seed)
        self.conv1 = nn.Conv2d(3, 6, 5),
        self.pool = nn.MaxPool2d(2, 2),
        self.conv2 = nn.Conv2d(6, 16, 5),
        self.fc1 = nn.Linear(16 * 5 * 5, 120),
        self.fc2 = nn.Linear(120, 84),
        self.fc3 = nn.Linear(84, n_labels)
    @staticmethod
    def conv_global_max_pool(x, conv):
        return F.relu(conv(x).transpose(1, 2).max(1)[0])
    
    def forward(self, x):
        x = self.embeddings(x).transpose(1, 2) 
        x = [self.conv_global_max_pool(x, conv) for conv in self.convs]
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc(x))
        x = self.output(x)
        return x

# Exploración de hiperparámetros: red convolucional (CNN)

# Experimento con mlflow: red convolucional (CNN)

In [22]:
F_COUNT = [50, 100, 200, 400]
F_LENGTH = [[2,3,4], [3, 4, 5], [4, 6, 8]]
d = [0.1, 0.3, 0.6]
l = [1e-3, 1e-6, 1e-9]
w = [1e-5, 1e-3, 1e-7]
  
for i in trange(20):
  FILTERS_COUNT = int(np.random.choice(F_COUNT, 1)) 
  FILTERS_LENGTH = F_LENGTH[int(np.random.choice([0], 1))]
  dropout = float(np.random.choice(d, 1))
  lr = float(np.random.choice(l, 1))
  wd = float(np.random.choice(w, 1))
  s = i

  mlflow.set_experiment("Random_search_hiperparámetros_CNN")
  with mlflow.start_run():
      mlflow.log_param("model_name", "CNN")
      mlflow.log_param("freeze_embedding", True)
      mlflow.log_params({
          "embedding_size": 50,
          "FILTERS_COUNT": FILTERS_COUNT,
          "FILTERS_LENGTH": FILTERS_LENGTH,
          "dropout": dropout,
          "regularización": lr,
          "weight decay": wd,
          "seed": s
      })
      model = MeliChallengeClassifier_CNN("./data/SBW-vectors-300-min5.txt", 
                                      preprocess.dictionary, 
                                      len(preprocess.target_to_idx) + 1,
                                      300, 
                                      True,
                                      seed=s
                                      )
      device = torch.device("cuda") 
      model = model.to(device)
      loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
      for epoch in trange(2):
          model.train()
          running_loss = []
          for idx, batch in enumerate(tqdm(train_loader)):
              optimizer.zero_grad()
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              loss_value = loss(output, target)
              loss_value.backward()
              optimizer.step()
              running_loss.append(loss_value.item())        
          mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
          
          model.eval()
          running_loss = []
          targets = []
          predictions = []
          for batch in tqdm(validation_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              running_loss.append(
                  loss(output, target).item()
              )
              targets.extend(batch["target"].numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
      
      with tempfile.TemporaryDirectory() as tmpdirname:
          targets = []
          predictions = []
          for batch in tqdm(test_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              targets.extend(target.cpu().detach().numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
              f"{tmpdirname}/predictions.csv.gz", index=False
          )
          mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)
          mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/20 [00:00<?, ?it/s]

INFO: 'Random_search_hiperparámetros_CNN' does not exist. Creating a new experiment


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

...

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Volvemos a correr con los mejores hiperparámetros
F_COUNT = [50, 100, 200]
F_LENGTH = [[2,3,4], [3, 4, 5], [4, 6, 8]]
d = [0.1, 0.3, 0.6]
lr = [1e-3, 1e-6, 1e-9]
w = [1e-5, 1e-3, 1e-7]
  
for i in trange(1):
  FILTERS_COUNT = int(np.random.choice(F_COUNT, 1)) 
  FILTERS_LENGTH = F_LENGTH[int(np.random.choice([0], 1))]
  dropout = float(np.random.choice(d, 1))
  lr = float(np.random.choice(l, 1))
  wd = float(np.random.choice(w, 1))
  s = 3

  mlflow.set_experiment("Experimento_CNN")
  with mlflow.start_run():
      mlflow.log_param("model_name", "CNN")
      mlflow.log_param("freeze_embedding", True)
      mlflow.log_params({
          "embedding_size": 50,
          "FILTERS_COUNT": FILTERS_COUNT,
          "FILTERS_LENGTH": FILTERS_LENGTH,
          "dropout": dropout,
          "regularización": lr,
          "weight decay": wd,
          "seed": s
      })
      model = MeliChallengeClassifier_CNN("./data/SBW-vectors-300-min5.txt", 
                                      preprocess.dictionary, 
                                      len(preprocess.target_to_idx) + 1,
                                      300, 
                                      True,
                                      seed=s
                                      )
      device = torch.device("cuda") 
      model = model.to(device)
      loss = nn.CrossEntropyLoss()
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
      for epoch in trange(5):
          model.train()
          running_loss = []
          for idx, batch in enumerate(tqdm(train_loader)):
              optimizer.zero_grad()
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              loss_value = loss(output, target)
              loss_value.backward()
              optimizer.step()
              running_loss.append(loss_value.item())        
          mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
          
          model.eval()
          running_loss = []
          targets = []
          predictions = []
          for batch in tqdm(validation_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              running_loss.append(
                  loss(output, target).item()
              )
              targets.extend(batch["target"].numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
      
      with tempfile.TemporaryDirectory() as tmpdirname:
          targets = []
          predictions = []
          for batch in tqdm(test_loader):
              data = batch["data"].to(device)
              target = batch["target"].type(torch.LongTensor).to(device)
              output = model(data)
              targets.extend(target.cpu().detach().numpy())
              predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
          pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
              f"{tmpdirname}/predictions.csv.gz", index=False
          )
          mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
          mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)
          mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/1 [00:00<?, ?it/s]

INFO: 'Experimento_CNN' does not exist. Creating a new experiment


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

  0%|          | 0/38245 [00:00<?, ?it/s]

  0%|          | 0/9562 [00:00<?, ?it/s]

# Generación de un archivo comprimido para mlflow

In [ ]:
!tar zcvf mlruns.tar.gz mlruns

mlruns/
mlruns/0/
mlruns/0/meta.yaml
mlruns/.trash/
mlruns/1/
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/metrics/
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/metrics/train_loss
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/metrics/validation_bacc
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/metrics/validation_loss
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/meta.yaml
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/artifacts/
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/artifacts/predictions.csv.gz
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/dropout
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/weight decay
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/freeze_embedding
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/model_name
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/regularización
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/params/embedding_size
mlruns/1/9d5affac72d54cbfb58682d4f3c0d0a4/tags/
mlru

In [ ]:
# %%bash

# # Una vez que lo descargan, lo descomprimen en su máquina con el siguiente comando
# tar xvf mlruns.tar.gz

# # Luego ejecutan en el lugar que lo descomprimieron lo siguiente
# mlflow ui